In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imp
import utils
imp.reload(utils)
%matplotlib inline

wd, train_set, test_set, label_map, inv_label_map, file_all, y_tr, test_file_all = utils.myinit(wd)

In [16]:
nnts = np.load('../preds/170620_01_pred_ts.npy')
nntr = np.load('../preds/170620_01_pred_tr.npy')

gbts = np.load('../preds/gbm_all_pred_ts_170624_01.npy')
gbtr = np.load('../preds/gbm_all_pred_tr_170624_01.npy')

wnnts = np.load('./pred/modelweightedNN1_date710_pred_test.npy')
wnntr = np.load('./pred/modelweightedNN1_date710_pred_train.npy')

nbtr = np.load("./pred/modelnn_boost_date716_pred_train.npy")
nbts = np.load("./pred/modelnn_boost_date716_pred_test.npy")

rnnts = np.load('./pred/modelresnet1_date715_pred_test.npy')
rnntr = np.load('./pred/modelresnet1_date715_pred_train.npy')

sifts = np.load('./pred/modelsiftgdbt_date719_pred_test.npy')
siftr = np.load('./pred/modelsiftgdbt_date719_pred_train.npy')

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold

y_ts = np.zeros((gbts.shape[0], 17))
p_tr = np.zeros((gbtr.shape[0], 17))

for i_c in range(17):
    skf = StratifiedKFold(n_splits=5)
    x_tr = np.c_[nntr[:, i_c], gbtr[:, i_c], wnntr[:, i_c], nbtr[:, i_c], rnntr[:, i_c], siftr[:, i_c]]
    x_ts = np.c_[nnts[:, i_c], gbts[:, i_c], wnnts[:, i_c], nbts[:, i_c], rnnts[:, i_c], sifts[:, i_c]]
    k_now = 0
    for i_tr, i_vl in skf.split(x_tr, y_tr[:, i_c]):
        lr = LogisticRegression(class_weight='balanced')
        lr.fit(x_tr[i_tr, :], y_tr[i_tr, i_c])
        p_tr[i_vl, i_c] = lr.predict_proba(x_tr[i_vl, :])[:,1]

    lr = LogisticRegression(class_weight='balanced')
    lr.fit(x_tr, y_tr[:, i_c])
    y_ts[:, i_c] = lr.predict_proba(x_ts)[:, 1]

In [25]:
th1 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(0, 1, 0.01))
th2 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(1, 0, -0.01))
th = (th1 + th2) / 2.0
utils.f2_score(y_tr, p_tr, th, num_classes=17)

0.92007698640649627

In [24]:
imp.reload(utils)
_th1 = utils.f2_opti_score_new(y_tr, p_tr, thresholds = np.arange(0, 1, 0.01))
_th2 = utils.f2_opti_score_new(y_tr, p_tr, thresholds = np.arange(1, 0, -0.01))
_th = (_th1 + _th2) / 2.0
utils.f2_score(y_tr, p_tr, _th, num_classes=17)

J:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.92117673951585888

In [27]:
imp.reload(utils)
submit_df = utils.to_submit(y_ts, th, test_set, inv_label_map)
submit_df.to_csv('./submit/2model_stacking_date719_no4.csv', index=False)

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

y_ts = np.zeros((gbts.shape[0], 17))
p_tr = np.zeros((gbtr.shape[0], 17))

for i_c in range(17):
    x_tr = np.c_[nntr[:, i_c], gbtr[:, i_c], wnntr[:, i_c], nbtr[:, i_c], rnntr[:, i_c], siftr[:, i_c]]
    x_ts = np.c_[nnts[:, i_c], gbts[:, i_c], wnnts[:, i_c], nbts[:, i_c], rnnts[:, i_c], sifts[:, i_c]]
    parameters = {'n_neighbors': range(3, 61, 3)}
    neigh = KNeighborsClassifier()
    clf = GridSearchCV(neigh, parameters, n_jobs=4)
    clf.fit(x_tr, y_tr[:, i_c])
    print('i: {}, Best score: {}, Best param {}.'.format(i_c, clf.best_score_, clf.best_params_['n_neighbors']))
    neigh = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'])
    neigh.fit(x_tr, y_tr[:, i_c])
    y_ts[:, i_c] = neigh.predict_proba(x_ts)[:, 1]

i: 0, Best score: 0.9562489191926679, Best param 54.
i: 1, Best score: 0.9723807406309445, Best param 51.
i: 2, Best score: 0.9647471528446849, Best param 39.
i: 3, Best score: 0.983423503545048, Best param 42.
i: 4, Best score: 0.9703797030559055, Best param 57.
i: 5, Best score: 0.9037278588897947, Best param 57.
i: 6, Best score: 0.9289508140023222, Best param 57.
i: 7, Best score: 0.9262333555670842, Best param 54.
i: 8, Best score: 0.9203784678475259, Best param 51.
i: 9, Best score: 0.9500975814619926, Best param 57.
i: 10, Best score: 0.9797919909088664, Best param 57.
i: 11, Best score: 0.9920699622026236, Best param 30.
i: 12, Best score: 0.9965414165369698, Best param 15.
i: 13, Best score: 0.9918970330294721, Best param 12.
i: 14, Best score: 0.9948368289730477, Best param 9.
i: 15, Best score: 0.998320116603671, Best param 15.
i: 16, Best score: 0.997653104078658, Best param 24.


In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

y_ts = np.zeros((gbts.shape[0], 17))
p_tr = np.zeros((gbtr.shape[0], 17))

n_neighbors = [54, 51, 39, 42,57,57,57,54,51,57,57,30,15,12,9,15,24]
for i_c in range(17):
    skf = StratifiedKFold(n_splits=5)
    x_tr = np.c_[nntr[:, i_c], gbtr[:, i_c], wnntr[:, i_c], nbtr[:, i_c], rnntr[:, i_c]]
    x_ts = np.c_[nnts[:, i_c], gbts[:, i_c], wnnts[:, i_c], nbts[:, i_c], rnnts[:, i_c]]
    k_now = 0
    for i_tr, i_vl in skf.split(x_tr, y_tr[:, i_c]):
        neigh = KNeighborsClassifier(n_neighbors=n_neighbors[i_c])
        neigh.fit(x_tr[i_tr, :], y_tr[i_tr, i_c])
        p_tr[i_vl, i_c] = neigh.predict_proba(x_tr[i_vl, :])[:,1]
        
    neigh = KNeighborsClassifier(n_neighbors=n_neighbors[i_c])
    neigh.fit(x_tr, y_tr[:, i_c])
    y_ts[:, i_c] = neigh.predict_proba(x_ts)[:, 1]

In [30]:
th1 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(0, 1, 0.01))
th2 = utils.f2_opti_score(y_tr, p_tr, thresholds = np.arange(1, 0, -0.01))
th = (th1 + th2) / 2.0
utils.f2_score(y_tr, p_tr, th, num_classes=17)

J:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.92450062098143981

In [32]:
submit_df = utils.to_submit(y_ts, th, test_set, inv_label_map)
submit_df.to_csv('./submit/6model_stackingwithknn_date719_no4.csv', index=False)

In [12]:
th * 0.95

array([ 0.16625,  0.133  ,  0.14725,  0.09025,  0.1805 ,  0.16625,
        0.15675,  0.20425,  0.19475,  0.21375,  0.171  ,  0.1995 ,
        0.2755 ,  0.3705 ,  0.7125 ,  0.114  ,  0.2565 ])